In [1]:
# Import installed modules
import pandas as pd
import numpy as  np
import imageio
from tqdm import tqdm

# Import the Python script func from the auxiliary folder
import sys
sys.path.insert(1, '../_auxiliary')
import func

In [2]:
# Set a local download path and the URL to the 67P shape model data set
DL_PATH = 'data/'
DL_URL = 'https://repos.cosmos.esa.int/socci/projects/SPICE_KERNELS/repos/rosetta/raw/kernels/' \
          + 'dsk/ROS_CG_M001_OSPCLPS_N_V1.OBJ'

# Download the shape model, create (if needed) the download path and store the data set
func.download_file(DL_PATH, DL_URL)

In [3]:
# Load the shape model. The first column lists whether the row is a vertex or face. The second,
# third and fourth column list the coordiantes (vertex) and vertex indices (faces)
COMET_67P_SHAPE_OBJ = pd.read_csv('data/ROS_CG_M001_OSPCLPS_N_V1.OBJ', delim_whitespace=True, \
                                  names=['TYPE', 'X1', 'X2', 'X3'])

# Print some shape model information
print('Some statistics and parameters of the shape model')
print(f'Rows and columns of the data set: {COMET_67P_SHAPE_OBJ.shape}')
print(f'Number of vertices: {COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ["TYPE"]=="v"].shape[0]}')
print(f'Number of faces: {COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ["TYPE"]=="f"].shape[0]}')
print('\n')

Some statistics and parameters of the shape model
Rows and columns of the data set: (1791747, 4)
Number of vertices: 597251
Number of faces: 1194496




In [4]:
# Print some examplarily extractions from the vertex and face sub set
print('Vertices (Sample)')
print(f'{COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ["TYPE"]=="v"].head()}')
print('Faces (Sample)')
print(f'{COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ["TYPE"]=="f"].head()}')
print('\n')

Vertices (Sample)
  TYPE        X1        X2        X3
0    v  0.570832 -1.000444  0.532569
1    v  0.564360 -1.000224  0.525360
2    v  0.557853 -0.997863  0.520762
3    v  0.553592 -0.998414  0.512192
4    v  0.550212 -0.992514  0.507304
Faces (Sample)
       TYPE      X1      X2      X3
597251    f   474.0   522.0   305.0
597252    f   474.0   719.0   522.0
597253    f  1961.0  2160.0  1651.0
597254    f  1961.0  2159.0  2160.0
597255    f  5513.0  5668.0  5285.0




In [5]:
# Assign the VERTICES and faces
VERTICES = COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ['TYPE'] == 'v'][['X1', 'X2', 'X3']].values \
               .tolist()
faces = COMET_67P_SHAPE_OBJ.loc[COMET_67P_SHAPE_OBJ['TYPE'] == 'f'][['X1', 'X2', 'X3']].values

In [6]:
# Print the minimum and maximum vertex indices in the face sub set
print(f'Minimum vertex index in faces: {np.min(faces)}')
print(f'Maximum vertex index in faces: {np.max(faces)}')
print('\n')

Minimum vertex index in faces: 1.0
Maximum vertex index in faces: 597251.0




In [7]:
# The index in the faces sub set starts at 1. For Python, it needs to start at 0.
faces = faces - 1

# Convert the indices to integer
faces = faces.astype(int)

# Convert the numpy array to a Python list
faces = faces.tolist()

In [8]:
# Now we need to define a main window class that is needed to set a window size / resolution.
# Based on the QT4 example:
# https://github.com/almarklein/visvis/blob/master/examples/embeddingInQt4.py
from PyQt5.QtWidgets import QWidget, QHBoxLayout

# Import visvis
import visvis as vv

# Define the class
class MainWindow(QWidget):
    def __init__(self, *args):
        QWidget.__init__(self, *args)
        self.fig = vv.backends.backend_pyqt5.Figure(self)
        self.sizer = QHBoxLayout(self)
        self.sizer.addWidget(self.fig._widget)
        self.setLayout(self.sizer)
        self.setWindowTitle('Rosetta')
        self.show()

In [9]:
# Create visvis application
app = vv.use()
app.Create()

# Create main window frame and set a resolution.
main_w = MainWindow()
main_w.resize(1200, 800)

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh's surface in this case
vv.mesh(vertices=VERTICES, faces=faces, verticesPerFace=3)

# Get axes objects
axes = vv.gca()

# Set a black background
axes.bgcolor = 'black'

# Deactivate the grid and make the x, y, z axes invisible
axes.axis.showGrid = False
axes.axis.visible = False

# Set some camera settings
# Please note: if you want to "fly" arond the comet with w, a, s, d (translation) and i, j, k, l
# (tilt) replace '3d' with 'fly'
axes.camera = '3d'

# Field of view in degrees
axes.camera.fov = 60

# Set default azmiuth and elevation angle in degrees
axes.camera.azimuth = 120
axes.camera.elevation = 25

# ... and run the application!
app.Run()

In [10]:
# Now let's create an animation

# Create visvis application
app = vv.use()
app.Create()

# Create main window frame and set a resolution.
main_w = MainWindow()
main_w.resize(500, 400)

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh's surface in this case
shape_obj = vv.mesh(vertices=VERTICES, faces=faces, verticesPerFace=3)
shape_obj.specular = 0.0
shape_obj.diffuse = 0.9

# Get figure
figure = vv.gcf()

# Get axes objects and set figure parameters
axes = vv.gca()
axes.bgcolor = (0, 0, 0)
axes.axis.showGrid = False
axes.axis.visible = False

# Set camera settings
#
axes.camera = '3d'
axes.camera.fov = 60
axes.camera.zoom = 0.1

# Turn off the main light
axes.light0.Off()

# Create a fixed light source
light_obj = axes.lights[1]
light_obj.On()
light_obj.position = (5.0, 5.0, 5.0, 0.0)

# Empty array that contains all images of the comet's rotation
comet_images = []

# Rotate camera in 300 steps in azimuth
for azm_angle in tqdm(range(300)):

    # Change azimuth angle of the camera
    axes.camera.azimuth = 360 * float(azm_angle) / 300

    # Draw the axes and figure
    axes.Draw()
    figure.DrawNow()

    # Get the current image
    temp_image = vv.getframe(vv.gca())

    # Apped the current image in 8 bit integer
    comet_images.append((temp_image*255).astype(np.uint8))

# Save the images as an animated GIF
imageio.mimsave('Comet67P.gif', comet_images, duration=0.04)

100%|██████████| 300/300 [00:10<00:00, 27.90it/s]
